In [ ]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
import time

import re
import numpy as np
import pandas as pd
df=pd.DataFrame()

URL= "https://www.hktvmall.com/hktv/en/main/search?page=0&q=%3Arelevance%3Azone%3Abeautynhealth%3Astreet%3Amain%3A"
URL1="https://www.hktvmall.com/hktv/en/main/search?page="
URL2="&q=%3Arelevance%3Azone%3Abeautynhealth%3Astreet%3Amain%3A"

driver = webdriver.Chrome(executable_path="/Applications/chromedriver")
driver.get(URL)
time.sleep(5)
html = driver.page_source
soup=BeautifulSoup(html, "html.parser")
URL_iter=0

while int(URL_iter)<867:
    for item in soup.find_all(class_="info-wrapper"):
    
        try:
            p_raw=item.find(class_="brand-product-name").text
            print("Raw Info:", p_raw)
        except:
            p_raw=str(np.nan)
            print("Raw Info:", p_raw)

        try:
            p_brand=re.match("^(.*?)\s\-\s", p_raw).group(1)
            print("Product Brand:", p_brand)
        except:
            p_brand=p_raw
            print("Product Brand:", p_brand)

        try:
            p_item=re.match("^(?:.*)\s\-\s(.*)", p_raw).group(1)
            print("Product Name:", p_item)

        except:
            p_item=p_raw
            print("Product Name:", p_item)

        try:
            p_sales=item.find(class_="salesNumber-container").text.replace("+ Sold", "")
            print("Sales:", p_sales)
        except:
            p_sales=np.nan
            print("Sales:", p_sales)

        try:
            p_oprice=item.find(class_="promotional").span.text
            p_oprice2=re.match("^(?:.*\$)(.*)", p_oprice).group(1)
            print("Original Price:", p_oprice2)
        except:
            p_oprice=np.nan 
            print("Original Price:", p_oprice)       

        try:
            p_dprice=item.find(class_="price").span.text.replace("$ ", "")
            print("Discount Price:", p_dprice)
        except:
            p_dprice=np.nan
            print("Discount Price:", p_dprice)

        try:
            promo=item.find(class_="deliveryLabelText").text
            print("Promo Tag:", promo)
        except:
            promo=np.nan
            print("Promo Tag:", promo)

        try:
            rating=item.find(class_="star_container")["data-rating"]
            print("Rating:", rating)
        except:
            rating=np.nan
            print("Rating:", rating)

        try:
            review=item.find(class_="review-number").text.replace("(", "").replace(")", "")
            print("Review:", review)
        except:
            review=np.nan
            print("Review:", review)

        try:
            vendor=item.find(class_="store-name-label").text
            print("Vendor:", vendor)
        except:
            vendor=np.nan
            print("Vendor:", vendor)

        try:
            category=item.find(class_="sepaButton add-to-cart-button")["data-category"]
            print("Product Category Code:", category)
        except:
            category=str(np.nan)
            print("Product Category Code:", category)

        print("\n")
        df=df.append({"Raw Info":p_raw, "Product Brand":p_brand, "Product Name":p_item, "Sales":p_sales, "Original Price":p_oprice2, "Discount price":p_dprice, "Promo Tag":promo, "Rating":rating, "Review":review, "Vendor":vendor, "Product Category Code":category}, ignore_index=True)
    
    URL_no=re.findall("\?page\=(\d*)\&q\=.*",URL)
    URL_iter=str(int(URL_no[0])+1)

    URL = URL1+URL_iter+URL2
    nextlink=URL1+URL_iter+URL2
    print(nextlink)

    subhtml=driver.get(nextlink)
    time.sleep(5)
    subhtml=driver.page_source
    soup=BeautifulSoup(subhtml, "html.parser")

# df.to_csv("hktvmall_skincare_females_and_males.csv")